## Basic problem

We are interested in a solution to
$$
\begin{equation}
\begin{cases}
    c\frac{\partial P}{\partial t} + \vec{\nabla}\vec{q} = f\text{ in }\Omega \\
    \vec{q} = -a\vec\nabla P
\end{cases}
\label{eq:cfd_problem} \tag{1}
\end{equation}
$$

with Dirichlet ($P = P_e$ on $\partial\Omega_D$) and Neumann ($-a\frac{\partial P}{\partial \vec\eta} = g$ on $\partial\Omega_N$) boundary conditions.

We want, given a scalar function $g$ depending on a solution to this problem, to find its sensitivity with respect to $a$.

We will solve problem __(1)__ numerically by use of the LMHFE memthod.
This means that $\Omega$ will be subdivided into elements (cells), and our values will be interpreted as vectors: $\vec P$, $\vec a$ and $\vec c$ with $i$-th elements representing average values of $P$, $a$ and $c$ over $i$-th cell.

In these terms, $g$ becomes $g(\vec P)$ with sensitivity $\frac{dg}{d\vec a}$.

$$
\frac{dg}{d\vec a} = \frac{\partial g}{\partial \vec a} + \frac{\partial g}{\partial \vec P}\frac{d \vec P}{d \vec a}
$$

From here and forward, we will denote full derivatives as $\frac{dg}{d\vec a} \equiv g_{\vec a}$, and partial derivatives as $\frac{\partial g}{\partial \vec a} \equiv \partial g_{\vec a}$.

$$
g_{\vec a} = \partial g_{\vec a} + \partial g_{\vec P} \vec P_{\vec a}
$$

<font color=red>
Let's assume $g$ has no explicit dependence on $a$.
This will elmiminate the first term from this sum, leaving us with just
$$
g_{\vec a} = \partial g_{\vec P}\vec P_{\vec a}
$$
</font>

We also asssume that since the analytical form for $g$ is known, $\partial g_{\vec P}$ is also dervied and known.

What we're interested in is sensitivity $\vec P_{\vec a}$.
It comes from a method used to obtain $\vec P$.

LMHFE (and MHFE) use recurrent relation to find $\vec P$ on the next time step.
They also use $P_T$ &mdash; a solution's average over the cell edge.
Let's combine all of these values in a vector $\vec P_T$ (note that an edge could have one or two neighbouring cells). For $i$-th cell, solution $P^{[n + 1]}_i$ on step $n + 1$ is given by:

$$
P^{[n + 1]}_i = \frac{a_i}{\beta_i} \sum\limits_{\text{edge }j\in\text{ boundary of }i} \alpha_{ij} P_{T,j}^{[n+1]} + \frac{\lambda_i}{\beta_i}P_i^{[n]} + \frac{Q_{S,i}}{\beta_i}
$$
where $\alpha_{ij}$ is a per-edge coefficient derived from geometry and
$$
\lambda_i = \frac{c_i\left|A_i\right|}{\tau}, \alpha_i = \sum\limits_{j} \alpha_{ij}, \beta_i = \lambda_i + a_i\alpha_i
$$

<font color=red>
$Q_{S,i}$ is proportional to a mean value of $f$ over $i$-th cell and is, in our case, zero.
$$
Q_{S,i} = 0
$$
</font>

The equation could be rewritten in a vector form as:
$$
\vec P^{[n+1]} = U \vec P_T^{[n+1]} + V \vec P^{[n]}
$$

$U_{ij} = \frac{a_i\alpha_{ij}}{\beta_i}$ if $j \in \partial_i$ and $0$ otherwise.

$V_{ii} = \frac{\lambda_i}{\beta_i}$, $V_{ij} = 0$ if $i \ne j$.

Sensitivity $\frac{dg}{da}$ on time step $n + 1$ then becomes
$$
g_{\vec a}^{[n]} = \partial g_{\vec a}^{[n]} + \partial g_{\vec P}^{[n]} \vec P_{\vec a}^{[n]}
$$
where
$$
\vec P_{\vec a}^{[n]} = \left( U\vec P_T^{[n]} \right)_\vec a + V\vec P^{[n-1]}_{\vec a}
$$
since $V_{\vec a} = 0$.

$\vec P_{\vec a}^{[0]} = 0$, so:
$$
\vec P_{\vec a}^{[n]} = \sum\limits_{i = 1}^{n}V^{n-i}\left(U\vec P_T^{[i]}\right)_\vec a
$$

$$
\left( \left( U\vec P_T^{[i]}\right)_\vec a \right)_{ij}
= \frac{d\left(U\vec P_T^{[i]}\right)_i}{da_j}
= \frac{d}{da_j} \left( \sum\limits_k U_{ik} \left( \vec P_T^{[i]} \right)_k \right)
$$
$$
\left( \left( U\vec P_T^{[i]}\right)_\vec a \right)_{ij}
= \sum\limits_k \left(\frac{d}{da_j}U_{ik}\right)\left( \vec P_T^{[i]} \right)_k + \sum\limits_k U_{ik} \frac{d}{da_j}\left( \vec P_T^{[i]} \right)_k
$$
$$
\left( \left( U\vec P_T^{[i]}\right)_\vec a \right)_{ij}
= \sum\limits_k \left(\frac{d}{da_j}U_{ik}\right)\left( \vec P_T^{[i]} \right)_k + \left(U\left(\vec P_T^{[i]}\right)_{\vec a}\right)_{ij}
$$

Therefore
$$
\left( U\vec P_T^{[i]} \right)_{\vec a}
= \left[ \vec D_1 \biggr| \vec D_2 \biggr| ... \biggr| \vec D_j \biggr| ... \right] + U\left(\vec P_T^{[i]}\right)_{\vec a}
\equiv \mathbf D + U\left(\vec P_T^{[i]}\right)_{\vec a}
$$
where $\vec D_j = U_{a_j}\vec P_T^{[i]}$.

Since $U_{ij} = \frac{a_i\alpha_{ij}}{\beta_i}$ for $j \in \partial_I$, then $\left( U_{a_k} \right)_{ij} =$
* $0$ if $i \ne k$ or $j \notin \partial_i$
* $\frac{\alpha_{kj}}{\beta_k}$ otherwise

&mdash; $U_{a_k}$ only contains $k$-th row, which will make $\mathbf D$ diagonal.

## LMHFE

Im lumped MHFE, $\vec P_T^{[i]}$ is obtained from:
$$
M^{[i]}\vec P_T^{[i]} = \vec b^{[i]}
$$

For MHFE and LMHFE, $\vec b^{[i]}$ is calculated from different values, that's why here <font color=red>we are only going to look at LMHFE</font>.

In LMHFE
$$
\vec b^{[i]} = \vec b\left(\vec P_T^{[i-1]}\right)
$$
and $M^{[i]}$ does not depend on previous solutions.

$$
\left( M^{[i]}\vec P_T^{[i]} \right)_{\vec a} = \left( \vec b \left( \vec P_T^{[i-1]}\right)\right)_{\vec a}
$$

Like in (...):
$$
\left( M^{[i]}\vec P^{[i]}_T \right)_{\vec a}
= \left[ \vec F^{[i]}_1 \bigg| ... \bigg| \vec F^{[i]}_j \bigg| ... \right] + M\left(\vec P_T^{[i]}\right)_{\vec a}
\equiv \mathbf{F}^{[i]} +M\left(\vec P_T^{[i]}\right)_{\vec a}
$$
where $\vec F^{[i]}_j = M_{a_j}\vec P_T^{[i]}$.

$\vec b$ is simpler:
$$
\left(\vec b\left(\vec P_T^{[i]}\right)\right)_{\vec a}
= \partial \vec b^{[i]}_{\vec a} + \partial \vec b^{[i]}_{\vec P_T^{[i - 1]}}\left(\vec P_T^{[i-1]}\right)_{\vec a}
$$

Which means:
$$
M^{[i]}\left( \vec P_T^{[i]}\right)_{\vec a}
= \partial \vec b^{[i]}_{\vec a} + \partial \vec b^{[i]}_{\vec P_T^{[i - 1]}}\left(\vec P_T^{[i-1]}\right)_{\vec a} - \mathbf F^{[i]}
$$

### Evaluate $M$ derivatives

If $M = \left( m_{ij} \right)$ then its diagonal terms are:

$$
m_{ii}
= m^A_{ii} + m^B_{ii}
,
m^A_{ii}
= a_A\left(\left(B_{ii}^{-1}\right)_A - \frac{\alpha_{Ai}^2}{\alpha_A}\right) + \frac{1}{n_f} \frac{c_A\|A\|}{\Delta t}$$

where $A$ and $B$ are triangles adjacent to the $i$-th edge, and its off-diagonal terms are:

$$ m_{ij} = a_A \left( \left(B^{-1}_{ij}\right)_A - \frac{\alpha_{Ai}\alpha_{Aj}}{\alpha_A} \right) $$

$M_{a_k}$ therefore will only contain elements corresponding to edges $i$, $j$ that are part of border $\partial_k$ of cell $k$:

$\left( M_{a_k} \right)_{ij} =$
* $0$ if edges $i,j \notin \partial_k$
* $2 \left( \left(B_{ii}^{-1}\right)_k - \frac{\alpha_{ki}^2}{\alpha_k} \right)$ if $i = j$ and $i \in \partial_k$
* $\left( \left(B_{ij}^{-1}\right)_k - \frac{\alpha_{ki}\alpha_{kj}}{\alpha_k} \right)$ if $i \ne j$ and $i,j \in \partial_k$

This way, $M_{a_k}$ is a very sparse matrix.

_Note that $M^{[i]}$ depends on time step $i$ only when using variable time step size.
Derivatives of $M^{[i]}$ are not time-dependant._

### Evaluate $\vec b$ derivatives

$$
\left( \vec b^{[i]} \right)_k = \frac{c\left(\|A_m\| + \|A_n\|\right)\left(\vec P_T^{[i-1]}\right)_k}{n_f \Delta t}
$$
where $m$, $n$ contain $k$-th edge in their border.

It's easy to see that
$$
\partial \vec b_{\vec a} = 0
$$
and that
$$
\left( \partial \vec b^{[i]}_{\vec P_T^{[i-1]}} \right)_{ii}
= \frac{c\left(\|A_m\| + \|A_n\|\right)}{n_f \Delta t}
$$
$$
\left( \partial \vec b^{[i]}_{\vec P_T^{[i-1]}} \right)_{ij}
= 0
$$
&mdash; $\partial \vec b^{[i]}_{\vec P_T^{[i-1]}}$ is a diagonal matrix.

### Iterative calculation

$$
\vec P_{\vec a}^{[n]}
= \sum\limits_{i = 1}^{n}V^{n-i}\left(U\vec P_T^{[i]}\right)_\vec a
= \left( U\vec P_T^{[n]}\right)_{\vec a} + V\sum\limits_{i=1}^{n-1}V^{n-1-i}\left(U\vec P_T^{[i]}\right)_{\vec a}
= \left( U\vec P_T^{[n]}\right)_{\vec a} + V \vec P_{\vec a}^{[n-1]}
$$

$$
g^{[n]}_{\vec a} = \partial g^{[n]}_{\vec P^{[n]}} \vec P^{[n]}_{\vec a} = \partial g^{[n]}_{\vec P^{[n]}} \left( \left( U\vec P_T^{[n]} \right)_{\vec a} + V\vec P^{[n-1]}_{\vec a}\right)
$$


## Variable in time $a$

When $a$ is a variable of time, the following modifications need to be made:
$$
\left(\frac{dg}{d\vec a}\right)^{[n]} = g^{[n]}_{\vec a} \rightarrow g^{[n]}_{\vec a^{[n]}}
$$
$$
\partial g^{[n]}_{\vec P^{[n]}}\vec P^{[n]}_{\vec a}
\rightarrow \partial g^{[n]}_{\vec P^{[n]}}\vec P^{[n]}_{\vec a^{[n]}}
= \partial g^{[n]}_{\vec P^{[n]}} \left( \left( U\vec P_T^{[n]}\right)_{\vec a^{[n]}} + V\vec P^{[n-1]}_{\vec a^{[n]}} \right)
$$

From the previous step, we will always know $\vec P^{[n-1]}_{\vec a^{[n-1]}}$.
From it, we will evaluate
$$
\vec P^{[n-1]}_{\vec a^{[n]}}
= \vec P^{[n-1]}_{\vec a^{[n-1]}} \vec a^{[n-1]}_{\vec a^{[n]}}
= \vec P^{[n-1]}_{\vec a^{[n-1]}} \vec a^{[n-1]}_t t_{\vec a^{[n]}}
$$

$\vec a^{[n-1]}_t$ and $t_{\vec a^{[n]}}$ are easy to evaluate: first is a column vector of time derivatives of elements of $\vec a$ at time $t^{[n-1]} = t^{[n]} - \tau$, and the second is a row vector of inverses of said detivatives at time $t^{[n]}$:
$$
\left(\vec a^{[n-1]}_{\vec a^{[n]}}\right)_{ij}
= \left( a^{[n-1]}_i \right)_{a^{[n]}_j}
= \frac{\left(a^{[n-1]}_i\right)_t}{\left(a^{[n]}_j\right)_t}
$$
This could be found analytically.

Same way
$$
M^{[i]}\left( \vec P_T^{[i]}\right)_{\vec a}
= \partial \vec b^{[i]}_{\vec a} + \partial \vec b^{[i]}_{\vec P_T^{[i - 1]}}\left(\vec P_T^{[i-1]}\right)_{\vec a} - \mathbf F^{[i]}
$$
converts to
$$
M^{[i]}\left( \vec P_T^{[i]}\right)_{\vec a^{[i]}}
= \partial \vec b^{[i]}_{\vec a^{[i]}} + \partial \vec b^{[i]}_{\vec P_T^{[i - 1]}}\left(\vec P_T^{[i-1]}\right)_{\vec a^{[i]}} - \mathbf F^{[i]}
$$
with $\vec F^{[i]}_j = M_{a_j^{[i]}}\vec P_T^{[i]}$.

And $\left( \vec P_T^{[i-1]} \right)_{\vec a^{[i]}}$ becomes $\left( \vec P_T^{[i-1]} \right)_{\vec a^{[i-1]}} \vec a^{[i-1]}_{\vec a^{[i]}}$.

## Algorithm

Let's denote: $N$ &mdash; number of cells and $K$ &mdash; number of cell edges.

In preparation, we should callculate and save:
1. $V$ ($N\times N$ matrix)
1. $\partial g_\vec P$ ($N$ row-vector)

For each time step we:
1. Perform LMHFE solution. From it we need:
  1. $M$ ($K\times K$ matrix)
  1. $P_T$ ($K$ column-vector)
1. From the previouis step we must preserve:
  1. $\vec P^{[i-1]}_{\vec a^{[i-1]}}$ ($N\times N$ matrix)
  1. $\left( \vec P_T^{[i-1]} \right)_{\vec a^{[i-1]}}$ ($K\times N$ matrix)
1. Calculate $U^{[i]}$ ($N\times K$ matrix; $U$ depends on $a$ and therefore is variable in time)
1. Calculate $U^{[i]}_{a^{[i]}}$ ($N$ $N\times K$ matrices)
1. Calculate $M_{a^{[i]}_j}$ ($N$ $K\times K$ matrices)
<!-- 1. Calculate $\partial b^{[i]}_{\vec a^{[i]}}$ ($K\times N$ matrix) -->
1. Calculate $\partial b^{[i]}_{\vec P_T^{[i-1]}}$ ($K\times K$ diagonal matrix)
1. Find $\mathbf F$
1. Find $\left(\vec P_T^{[i]} \right)_{\vec a^{[i]}}$ ($K\times N$ matrix). _Carries to the next step._
1. Find $\mathbf D$ ($K\times N$ matrix)
1. Find $\left( U\vec P_T^{[i]} \right)_{\vec a^{[i]}}$ ($N\times N$ matrix)
1. Find $\vec P^{[i]}_{\vec a^{[i]}}$ ($N\times N$ matirx). _Carries to the next step_
1. Find $g_{\vec a^{[i]}}$ ($N$ row-vector)